# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397815877140                   -0.90    5.2   27.6ms
  2   -8.400238361214       -2.62       -1.73    1.0   19.1ms
  3   -8.400402429978       -3.78       -2.95    1.5   19.9ms
  4   -8.400427746381       -4.60       -2.91    3.2   24.6ms
  5   -8.400427982463       -6.63       -3.09    1.0   19.0ms
  6   -8.400428147274       -6.78       -5.03    1.0   70.6ms
  7   -8.400428151913       -8.33       -4.51    3.2   26.2ms
  8   -8.400428152200       -9.54       -5.79    1.2   20.4ms
  9   -8.400428152209      -11.07       -6.23    2.0   22.6ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397818300903                   -0.90           5.0    770ms
  2   -8.400385642999       -2.59       -1.79   0.80    2.2    495ms
  3   -8.400423203899       -4.43       -3.02   0.80    1.0    207ms
  4   -8.400428109924       -5.31       -3.50   0.80    2.5   20.7ms
  5   -8.400428148927       -7.41       -4.85   0.80    1.0   16.4ms
  6   -8.400428152203       -8.48       -5.58   0.80    3.2   22.5ms
  7   -8.400428152209      -11.28       -6.45   0.80    1.0   16.8ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

┌ Warning: x_tol is deprecated. Use x_abstol or x_reltol instead. The provided value (-1) will be used as x_abstol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:110
┌ Warning: f_tol is deprecated. Use f_abstol or f_reltol instead. The provided value (-1) will be used as f_reltol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:120
n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.970021156820                   -1.05    3.52s
  2   -1.896490357628        0.46       -0.70    141ms
  3   -4.839677377880        0.47       -0.38   43.7ms
  4   -6.650325530993        0.26       -0.51   43.3ms
  5   -7.804558040399        0.06       -0.89   83.2ms
  6   -8.124406522898       -0.50       -1.26   32.2ms
  7   -8.248764968722       -0.91       -1.90   32.4ms
  8   -8.339042192453       -1.04       -1.93   32.7ms
  9   -8.363130122385       -1.62       -2.16   32.5ms
 10   -8.385733530526       -1.65       -2.38   55

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397838742750                   -0.90    5.5   28.3ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427980395                   -1.79    8.21s
  2   -8.400428152209       -6.76       -4.03    3.08s
  3   -8.400428152209      -14.45       -7.82    141ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.9748064101557237e-7
|ρ_newton - ρ_scfv| = 3.8982448433523255e-7
|ρ_newton - ρ_dm|   = 2.0268891439612135e-6
